# Assessing intersections of FUAs and admin boundaries

Urban boundaries are not tied to administrative boundaries, and often spill across borders. This notebook will explore how Functional Urban Areas (FUAs) intersect with admin boundries, through the lens of population density.

For each FUA, we will identify the intersecting admin bounds, and determine the percentage of its population in each intersection unit

In [ ]:
import sys
import os
import rasterio

import geopandas as gpd
import pandas as pd

sys.path.append("C:/WBG/Work/Code/GOSTrocks/src")

import GOSTrocks.dataMisc as dMisc
from GOSTrocks.misc import tPrint

from admin_pop_functions import *

%load_ext autoreload
%autoreload 2

In [ ]:
data_folder = "C:/WBG/Work/data"
ucdb_file = os.path.join(
    data_folder,
    "URBAN",
    "GHS_STAT_UCDB2015MT_GLOBE_R2019A",
    "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg",
)
fua_file = os.path.join(
    data_folder, "URBAN", "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
)

out_folder = "C:/WBG/Work/MENA_Urban/"
pop_res_folder = os.path.join(out_folder, "urban_data", "pop")
if not os.path.exists(pop_res_folder):
    os.makedirs(pop_res_folder)

pop_data = os.path.join(data_folder, "POP", "ppp_2020_1km_Aggregated.tif")
popR = rasterio.open(pop_data)

# Scenario vairables
adm_level = "ADM2"
fua = gpd.read_file(fua_file)
if fua.crs != popR.crs:
    fua = fua.to_crs(popR.crs)

In [ ]:
all_res = []
for lbl, curF in fua.groupby("Cntry_ISO"):
    pop_out_file = os.path.join(pop_res_folder, f"{lbl}_admin_urban_pop.csv")
    if not os.path.exists(pop_out_file):
        process = True
        try:
            cur_admin = dMisc.get_geoboundaries(lbl, adm_level)
            process = True
        except:
            tPrint(f"Could not find {adm_level} for {lbl}")
            process = False
        if process:
            if cur_admin.crs != popR.crs:
                cur_admin = cur_admin.to_crs(popR.crs)
            tPrint(f"Processing {lbl}")
            country_res = []
            for idx, row in curF.iterrows():
                city_res = calculate_admin_pop(row, cur_admin, popR)
                country_res.append(city_res)
            country_admin_res = pd.concat(country_res)
            country_admin_res.to_csv(pop_out_file)
            all_res.append(country_admin_res)
    else:
        tPrint(f"Already processed {lbl}")
        country_admin_res = pd.read_csv(pop_out_file, index_col=0)
        all_res.append(country_admin_res)

In [ ]:
complete_results = pd.concat(all_res)
complete_results.to_csv(os.path.join(pop_res_folder, "complete_results.csv"))